In [4]:
# Install dependencies
!pip install pdfplumber scikit-learn PyPDF2 numpy pytesseract Pillow
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [5]:
# Install dependencies
!pip install pdfplumber scikit-learn PyPDF2 numpy pytesseract Pillow

# Import libraries
from PyPDF2 import PdfReader, PdfWriter
import os
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import pdfplumber
import pytesseract
from PIL import Image
import io
import numpy as np

class ImprovedRealEstateDocumentProcessor:
    def __init__(self):
        # Enhanced keywords with more variations and synonyms
        self.keywords = {
            'deeds': [
                'title to real estate', 'quitclaim deed', 'warranty deed', 'general warranty deed',
                'special warranty deed', 'deed of trust', 'grant deed', 'bargain and sale deed',
                'grantee', 'grantor', 'conveyance', 'transfer of ownership', 'legal description',
                'habendum clause', 'consideration', 'acknowledgment', 'notarized'
            ],
            'property_cards': [
                'property card', 'parcel id', 'land value', 'assessed value', 'market value',
                'property assessment', 'tax assessment', 'square footage', 'lot size',
                'building description', 'property characteristics', 'zoning', 'tax map',
                'parcel number', 'tax id', 'real property', 'assessment roll'
            ],
            'tax_documents': [
                'tax bill', 'amount due', 'tax year', 'property tax', 'real estate tax',
                'tax assessment', 'tax collector', 'payment due', 'tax notice',
                'delinquent tax', 'tax lien', 'installment', 'tax payment',
                'municipal tax', 'county tax', 'school tax', 'fire tax'
            ]
        }

        # Enhanced patterns for document boundaries
        self.recording_stamps = [
            r'recorded on \w+ \d{1,2}, \d{4}',
            r'official records',
            r'book \d+ page \d+',
            r'instrument number',
            r'recorded \d{1,2}/\d{1,2}/\d{4}',
            r'filing date',
            r'document number',
            r'reception number'
        ]

        # Improved ML pipeline with better parameters
        self.model = Pipeline([
            ('tfidf', TfidfVectorizer(
                max_features=5000,
                ngram_range=(1, 3),  # Include bigrams and trigrams
                stop_words='english',
                min_df=2,
                max_df=0.8,
                lowercase=True
            )),
            ('clf', MultinomialNB(alpha=0.1))
        ])

    def extract_text_from_pdf(self, pdf_path):
        """Enhanced text extraction with multiple fallback methods"""
        text = ""

        # Method 1: Try pdfplumber first
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    if page.extract_text():
                        text += page.extract_text() + "\n"
                    else:
                        # Fallback to OCR for image-based pages
                        try:
                            img = page.to_image(resolution=300)
                            pil_img = img.original
                            ocr_text = pytesseract.image_to_string(pil_img)
                            text += ocr_text + "\n"
                        except:
                            pass
        except Exception as e:
            print(f"pdfplumber failed for {pdf_path}: {e}")

        # Method 2: Fallback to PyPDF2 if pdfplumber fails
        if not text.strip():
            try:
                reader = PdfReader(pdf_path)
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
            except Exception as e:
                print(f"PyPDF2 also failed for {pdf_path}: {e}")

        # Method 3: OCR as last resort for entire document
        if not text.strip():
            try:
                with pdfplumber.open(pdf_path) as pdf:
                    for page in pdf.pages:
                        try:
                            img = page.to_image(resolution=300)
                            pil_img = img.original
                            ocr_text = pytesseract.image_to_string(pil_img)
                            text += ocr_text + "\n"
                        except:
                            continue
            except:
                pass

        return text.strip()

    def extract_text_from_page_range(self, pdf_path, start_page, end_page):
        """Enhanced text extraction for page ranges"""
        text = ""

        try:
            with pdfplumber.open(pdf_path) as pdf:
                for i in range(start_page, end_page + 1):
                    if i < len(pdf.pages):
                        page = pdf.pages[i]
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text + "\n"
                        else:
                            # Try OCR for this page
                            try:
                                img = page.to_image(resolution=300)
                                pil_img = img.original
                                ocr_text = pytesseract.image_to_string(pil_img)
                                text += ocr_text + "\n"
                            except:
                                pass
        except Exception as e:
            print(f"Error extracting text from pages {start_page}-{end_page}: {e}")

        return text.strip()

    def preprocess_text(self, text):
        """Clean and preprocess text for better classification"""
        if not text:
            return ""

        # Remove extra whitespace and normalize
        text = re.sub(r'\s+', ' ', text.strip())

        # Remove non-ASCII characters that might cause issues
        text = ''.join(char for char in text if ord(char) < 128)

        # Normalize common variations
        text = re.sub(r'\b(deed|deeds)\b', 'deed', text.lower())
        text = re.sub(r'\b(tax|taxes)\b', 'tax', text.lower())
        text = re.sub(r'\b(property|properties)\b', 'property', text.lower())

        return text

    def train_model(self, training_dir):
        """Enhanced model training with validation"""
        texts, labels = [], []

        for doc_type in ['deeds', 'property_cards', 'tax_documents']:
            dir_path = os.path.join(training_dir, doc_type)
            if os.path.exists(dir_path):
                for filename in os.listdir(dir_path):
                    if filename.endswith('.pdf'):
                        try:
                            text = self.extract_text_from_pdf(os.path.join(dir_path, filename))
                            if text:
                                processed_text = self.preprocess_text(text)
                                if processed_text:
                                    texts.append(processed_text)
                                    labels.append(doc_type)
                                else:
                                    print(f"Warning: No processable text from {filename}")
                            else:
                                print(f"Warning: No text extracted from {filename}")
                        except Exception as e:
                            print(f"Error processing {filename}: {e}")
            else:
                print(f"Warning: Training directory not found: {dir_path}")

        if texts and len(set(labels)) > 1:
            self.model.fit(texts, labels)

            # Perform cross-validation to check model performance
            scores = cross_val_score(self.model, texts, labels, cv=min(5, len(texts)//2))
            print(f"Cross-validation scores: {scores}")
            print(f"Average CV score: {np.mean(scores):.3f}")

            # Print classification report
            predictions = self.model.predict(texts)
            print("\nTraining Classification Report:")
            print(classification_report(labels, predictions))

        else:
            print("Error: Insufficient training data found. Cannot train model.")

    def find_document_boundaries(self, pdf_path):
        """Enhanced boundary detection"""
        boundaries = []

        try:
            with pdfplumber.open(pdf_path) as pdf:
                full_text = ""
                for i, page in enumerate(pdf.pages):
                    page_text = page.extract_text() if page.extract_text() else ""
                    # If no text, try OCR
                    if not page_text.strip():
                        try:
                            img = page.to_image(resolution=300)
                            pil_img = img.original
                            page_text = pytesseract.image_to_string(pil_img)
                        except:
                            page_text = ""

                    full_text += page_text + f"\n---PAGE_BREAK---{i+1}---\n"

                # Look for recording stamps and other boundary markers
                for pattern in self.recording_stamps:
                    for match in re.finditer(pattern, full_text.lower()):
                        closest_page_break = full_text[:match.end()].rfind("---PAGE_BREAK---")
                        if closest_page_break != -1:
                            page_num_str = full_text[closest_page_break:].split("---PAGE_BREAK---")[1].split("---")[0]
                            boundaries.append(int(page_num_str) - 1)

                # Also look for other document separators
                additional_patterns = [
                    r'new document',
                    r'document \d+',
                    r'page 1 of \d+',
                    r'return to:'
                ]

                for pattern in additional_patterns:
                    for match in re.finditer(pattern, full_text.lower()):
                        closest_page_break = full_text[:match.end()].rfind("---PAGE_BREAK---")
                        if closest_page_break != -1:
                            page_num_str = full_text[closest_page_break:].split("---PAGE_BREAK---")[1].split("---")[0]
                            boundaries.append(int(page_num_str) - 1)

        except Exception as e:
            print(f"Error finding boundaries: {e}")

        if not boundaries:
            # Fallback: assume each page is a document
            try:
                with pdfplumber.open(pdf_path) as pdf:
                    return [(i, i) for i in range(len(pdf.pages))]
            except:
                return [(0, 0)]  # Single document fallback

        boundaries = sorted(list(set(boundaries)))
        doc_ranges = []
        start_page = 0

        for boundary_page in boundaries:
            if boundary_page > start_page:
                doc_ranges.append((start_page, boundary_page - 1))
            start_page = boundary_page

        # Add the last document
        try:
            with pdfplumber.open(pdf_path) as pdf:
                if start_page < len(pdf.pages):
                    doc_ranges.append((start_page, len(pdf.pages) - 1))
        except:
            pass

        return doc_ranges if doc_ranges else [(0, 0)]

    def classify_document(self, text):
        """Enhanced classification with confidence scoring"""
        if not text:
            return 'unknown'

        processed_text = self.preprocess_text(text)

        # Keyword-based classification with weighted scoring
        scores = defaultdict(float)

        for doc_type, terms in self.keywords.items():
            for term in terms:
                # Count occurrences and weight by term importance
                count = processed_text.lower().count(term.lower())
                if count > 0:
                    # Weight longer, more specific terms higher
                    weight = len(term.split()) * 1.5
                    scores[doc_type] += count * weight

        # Normalize scores
        total_score = sum(scores.values())
        if total_score > 0:
            for doc_type in scores:
                scores[doc_type] /= total_score

        # Check if keyword classification is confident enough
        if scores:
            predicted_type = max(scores.items(), key=lambda x: x[1])[0]
            confidence = scores[predicted_type]

            # Lower threshold for better recall
            if confidence > 0.3:
                return predicted_type

        # Fallback to ML model
        try:
            if hasattr(self.model, 'predict') and hasattr(self.model, 'predict_proba'):
                ml_prediction = self.model.predict([processed_text])[0]
                ml_probabilities = self.model.predict_proba([processed_text])[0]
                max_prob = max(ml_probabilities)

                # Only use ML prediction if confident enough
                if max_prob > 0.4:
                    return ml_prediction
        except Exception as e:
            print(f"ML prediction error: {e}")

        # Final fallback: use keyword classification even with low confidence
        if scores:
            return max(scores.items(), key=lambda x: x[1])[0]

        return 'unknown'

    def process_merged_pdf(self, pdf_path, output_dir):
        """Enhanced PDF processing with better error handling"""
        os.makedirs(output_dir, exist_ok=True)
        doc_ranges = self.find_document_boundaries(pdf_path)

        counts = defaultdict(int)

        try:
            reader = PdfReader(pdf_path)
            total_pages = len(reader.pages)
            print(f"Total pages in PDF: {total_pages}")
            print(f"Found {len(doc_ranges)} document ranges")

        except Exception as e:
            print(f"Error reading PDF: {e}")
            return

        for i, (start_page, end_page) in enumerate(doc_ranges):
            try:
                writer = PdfWriter()

                # Validate page range
                start_page = max(0, start_page)
                end_page = min(total_pages - 1, end_page)

                if start_page > end_page:
                    print(f"Skipping invalid page range: {start_page}-{end_page}")
                    continue

                # Add pages to writer
                for page_num in range(start_page, end_page + 1):
                    if page_num < total_pages:
                        writer.add_page(reader.pages[page_num])

                # Extract and classify text
                doc_text = self.extract_text_from_page_range(pdf_path, start_page, end_page)

                if doc_text:
                    doc_type = self.classify_document(doc_text)
                    counts[doc_type] += 1
                    filename = f"{doc_type.replace('_', '-')}_{counts[doc_type]}.pdf"
                    output_filepath = os.path.join(output_dir, filename)

                    # Save the document
                    with open(output_filepath, "wb") as f:
                        writer.write(f)

                    print(f"Saved {filename} (pages {start_page + 1}-{end_page + 1})")

                    # Print a sample of the extracted text for debugging
                    sample_text = doc_text[:200].replace('\n', ' ')
                    print(f"  Sample text: {sample_text}...")

                else:
                    print(f"Warning: No text extracted for document in range {start_page}-{end_page}")

                    # Still save the document but mark as unknown
                    counts['unknown'] += 1
                    filename = f"unknown_{counts['unknown']}.pdf"
                    output_filepath = os.path.join(output_dir, filename)

                    with open(output_filepath, "wb") as f:
                        writer.write(f)

                    print(f"Saved {filename} (pages {start_page + 1}-{end_page + 1}) - no text extracted")

            except Exception as e:
                print(f"Error processing document {i}: {e}")
                continue

        # Print summary
        print(f"\nProcessing Summary:")
        for doc_type, count in counts.items():
            print(f"  {doc_type}: {count} documents")

#  PATHS
train_path = "/content/drive/MyDrive/training_data"
merged_pdf_path = "/content/drive/MyDrive/merged_document/Sample_Full_Search_5.pdf"
output_dir = "/content/drive/MyDrive/real_estate_project/split_output"

# Run the improved processor
processor = ImprovedRealEstateDocumentProcessor()
processor.train_model(train_path)
processor.process_merged_pdf(merged_pdf_path, output_dir)

Cross-validation scores: [1. 1. 1. 1. 1.]
Average CV score: 1.000

Training Classification Report:
                precision    recall  f1-score   support

         deeds       1.00      1.00      1.00        50
property_cards       1.00      1.00      1.00        50
 tax_documents       1.00      1.00      1.00        50

      accuracy                           1.00       150
     macro avg       1.00      1.00      1.00       150
  weighted avg       1.00      1.00      1.00       150

Total pages in PDF: 70
Found 8 document ranges
Saved tax-documents_1.pdf (pages 1-7)
  Sample text: Date: November 30, 2021 Client: Cobb Dill & Hammett Attn: Accounting Department RE: Invoice for Abstracting Services Please see the invoice details below for title abstracting services. Please make ch...
Saved deeds_1.pdf (pages 8-8)
  Sample text: Administrator Login WELCOME REAL PROPERTY RECORD SEARCH REAL PROPERTY BILL SEARCH PERSONAL PROPERTY SEARCH MOTOR VEHICLE SEARCH CHECKOUT CONTACT US ( $0.00 C